In [1]:
import numpy as np
import h5py
import trimesh
import matplotlib.pyplot as plt
from random import random, uniform
import dlib
from scipy.optimize import lsq_linear
import pyglet 
import time
from trimesh.scene.viewer import SceneViewer
import sys

In [17]:
%run features/basel_features/index_loader.ipynb
%run basel_model/BaselModel.ipynb
%run optimizers/Optimizer.ipynb
%run optimizers/GradientDescentRefinement.ipynb
%run optimizers/AlternatingPoseShapeEstimation.ipynb
%run optimizers/AlternatingOptimizers.ipynb
%run optimizers/PoseEstimation.ipynb
%run optimizers/ShapeEstimation.ipynb
%run optimizers/OptimizerQueue.ipynb
%run input_generators/SingleImageInput.ipynb
%run input_generators/ContinuousImagesInput.ipynb
%run input_generators/VideoCameraInput.ipynb
%run output_handlers/PygletViewerHandler.ipynb

In [ ]:
baselModel = BaselModel('./model2017-1_bfm_nomouth.h5')
        
# get indexes of model feature points and associated dlib feature point numbers, indexed at 1
feature_point_indexes, dlib_points = get_feature_point_indexes('./feature_points.csv')
        
# extract only the feature points
shape_mean_features = baselModel.getShapeMean(pointIndexes=feature_point_indexes)
shape_pcaBasis_features = baselModel.getShapePCABasis(pointIndexes=feature_point_indexes)
        
triangles = baselModel.getModelMeshTriangles()

optimizers = OptimizerQueue()
optimizers.add(AlternatingOptimizers(
        mean = shape_mean_features,
        pcaBasis = shape_pcaBasis_features,
        pcaVariance = baselModel.getShapePCAVariance(),
        optimizers = [
            PoseEstimation(
                mean = shape_mean_features,
                pcaBasis = shape_pcaBasis_features,
                pcaVariance = baselModel.getShapePCAVariance(),
                numIterations = 1000
            ),
            ShapeEstimation(
                mean = shape_mean_features,
                pcaBasis = shape_pcaBasis_features,
                pcaVariance = baselModel.getShapePCAVariance()
            )
        ],
        numIterations = 1))
optimizers.add(GradientDescentRefinement(
        mean = shape_mean_features,
        pcaBasis = shape_pcaBasis_features,
        pcaVariance = baselModel.getShapePCAVariance(),
        numIterations = 500))

inputGenerator = VideoCameraInput(
        './shape_predictor_68_face_landmarks.dat',
        dlib_points,
        showVideo=True)

inputGenerator = ContinuousImagesInput(
        ['./test_images/test_image_2.jpeg'],
        './shape_predictor_68_face_landmarks.dat',
        dlib_points)


sceneViewer = SceneViewer(baselModel.getDefaultMeshScene(), start_loop=False)

def benchmark(dt):
    a, r, t, s = optimizers.optimizeNext(inputGenerator.getNext())
    setScene(baselModel.getMeshScene(a, r, t, s))

def setScene(newScene):
    newScene.set_camera()
    sceneViewer.scene = newScene
    sceneViewer.on_draw()
    
pyglet.clock.schedule(benchmark)

try:
    optimizers.openAll()
    pyglet.app.run()
except Exception as e:
    print(e)
    e.print_stack()
finally:
    inputGenerator.close()
    optimizers.closeAll()
